# 二値化画像作成アプリケーション
[OpenCV: GUI機能](http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_gui/py_table_of_contents_gui/py_table_of_contents_gui.html#py-table-of-content-gui)  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

## 準備

In [2]:
def get_th_rgb(img_blur,r1,g1,b1,r2,g2,b2):
    img_rgb = img_blur.copy()                            # rgbスケール
    rgb_min = np.array([r1,g1,b1], np.uint8)
    rgb_max = np.array([r2,g2,b2], np.uint8)
    th      = cv2.inRange(img_rgb, rgb_min, rgb_max)
    return th

In [3]:
def get_th_hsv(img_blur,h1,s1,v1,h2,s2,v2):
    img_hsv = cv2.cvtColor(img_blur, cv2.COLOR_RGB2HSV)  # hsvスケール
    hsv_min = np.array([h1,s1,v1], np.uint8)
    hsv_max = np.array([h2,s2,v2], np.uint8)
    th      = cv2.inRange(img_hsv, hsv_min, hsv_max)
    return th

In [4]:
def get_th_ad1(img_gray, Block_Size, C):
    th = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, Block_Size, C) # ブロック数, 引く数
    return th

In [5]:
def get_th(img, val):
    # グレースケール画像の準備
    img_blur   = cv2.GaussianBlur(img, (5,5), 0)
    img_gray   = cv2.cvtColor(img_blur, cv2.COLOR_BGR2GRAY)
    
    th1 = get_th_rgb(img_blur,val[0],val[1],val[2],val[3],val[4],val[5])   # rgb
    th2 = get_th_hsv(img_blur,val[6],val[7],val[8],val[9],val[10],val[11]) # hsv
    th3 = get_th_ad1(img_gray,2*val[12]+3,val[13])                             # ad

    th = cv2.bitwise_and(th1,th2)
    th = cv2.bitwise_and(th, th3)
    return th

In [6]:
STANDARD_SIZE = (512,300)

In [7]:
def nothing(x):
    pass

## Web カメラで実行

In [8]:
# Create a window
cv2.namedWindow('image')

# create trackbars for color change
trackbar_names = ['R1','G1','B1','R2','G2','B2','H1','S1','V1','H2','S2','V2','BS','C']
initial_values = [0,0,0,255,255,255,0,0,0,255,255,255,0,0]
for name,val in zip(trackbar_names,initial_values):
    cv2.createTrackbar(name,'image',val,255,nothing)

# create switch for ON/OFF functionality
switch = '0 : OFF \n1 : ON'
cv2.createTrackbar(switch,'image',0,1,nothing)

# 0 => IVcan, 1 => Webカメラ
cap = cv2.VideoCapture(1)


while(1):
    ret, img = cap.read()
    img      = cv2.resize(img, STANDARD_SIZE)

    # get current positions of four trackbars
    val_list = [cv2.getTrackbarPos(name,'image') for name in trackbar_names]
    s        = cv2.getTrackbarPos(switch,'image')

    if s == 0:
        th = img
    else:
        th = get_th(img, val_list)
        
    cv2.imshow('img',th)
    
    # キー入力を1ms待って、kが27(Esc)だったらbreakする
    k = cv2.waitKey(1)
    if k == 27:
        break
        

# キャプチャをリリースして、ウィンドウを全て閉じる
cap.release()
cv2.destroyAllWindows()

# 最後の値を表示
print(val_list[0:6])
print(val_list[6:12])
print([2*val_list[12]+3, val_list[13]])

[25, 60, 85, 150, 175, 170]
[95, 75, 90, 165, 170, 145]
[3, 255]


## 画像に対して実行

In [26]:
img = cv2.imread("data/target7.jpg")
img = cv2.resize(img,STANDARD_SIZE)
img_blur   = cv2.GaussianBlur(img, (5,5), 0)
img_gray   = cv2.cvtColor(img_blur, cv2.COLOR_BGR2GRAY)
th         = img.copy()

## RGB用

In [17]:
# Create a window
cv2.namedWindow('image')

# create trackbars for color change
cv2.createTrackbar('R1','image',0,255,nothing)
cv2.createTrackbar('G1','image',0,255,nothing)
cv2.createTrackbar('B1','image',0,255,nothing)
cv2.createTrackbar('R2','image',255,255,nothing)
cv2.createTrackbar('G2','image',255,255,nothing)
cv2.createTrackbar('B2','image',255,255,nothing)

# create switch for ON/OFF functionality
switch = '0 : OFF \n1 : ON'
cv2.createTrackbar(switch, 'image',0,1,nothing)

while(1):
    cv2.imshow('image',th)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

    # get current positions of four trackbars
    r1 = cv2.getTrackbarPos('R1','image')
    g1 = cv2.getTrackbarPos('G1','image')
    b1 = cv2.getTrackbarPos('B1','image')
    r2 = cv2.getTrackbarPos('R2','image')
    g2 = cv2.getTrackbarPos('G2','image')
    b2 = cv2.getTrackbarPos('B2','image')    
    s = cv2.getTrackbarPos(switch,'image')

    if s == 0:
        th = img.copy()
    else:
        th = get_th_rgb(img_blur,r1,g1,b1,r2,g2,b2)
        

cv2.destroyAllWindows()
print("{},{},{},{},{},{}".format(r1,g1,b1,r2,g2,b2))

0,0,0,255,255,255


## HSV用

In [10]:
# Create a window
cv2.namedWindow('image')

# create trackbars for color change
cv2.createTrackbar('H1','image',0,255,nothing)
cv2.createTrackbar('S1','image',0,255,nothing)
cv2.createTrackbar('V1','image',0,255,nothing)
cv2.createTrackbar('H2','image',255,255,nothing)
cv2.createTrackbar('S2','image',255,255,nothing)
cv2.createTrackbar('V2','image',255,255,nothing)

# create switch for ON/OFF functionality
switch = '0 : OFF \n1 : ON'
cv2.createTrackbar(switch, 'image',0,1,nothing)

while(1):
    cv2.imshow('image',th)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

    # get current positions of four trackbars
    h1 = cv2.getTrackbarPos('H1','image')
    s1 = cv2.getTrackbarPos('S1','image')
    v1 = cv2.getTrackbarPos('V1','image')
    h2 = cv2.getTrackbarPos('H2','image')
    s2 = cv2.getTrackbarPos('S2','image')
    v2 = cv2.getTrackbarPos('V2','image')    
    s = cv2.getTrackbarPos(switch,'image')

    if s == 0:
        th = img
    else:
        th = get_th_hsv(img_blur,h1,s1,v1,h2,s2,v2)
        

cv2.destroyAllWindows()
print("{},{},{},{},{},{}".format(h1,s1,v1,h2,s2,v2))

0,0,0,255,255,250


## Asaptive用

In [19]:
# Create a window
cv2.namedWindow('image')

# create trackbars for color change
cv2.createTrackbar('BS','image',0,255,nothing)
cv2.createTrackbar('C','image',0,30,nothing)
cv2.createTrackbar('Blur','image',0,50,nothing)

# create switch for ON/OFF functionality
switch = '0 : OFF \n1 : ON'
cv2.createTrackbar(switch, 'image',0,1,nothing)

while(1):
    cv2.imshow('image',th)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

    # get current positions of four trackbars
    bs = cv2.getTrackbarPos('BS','image')
    c  = cv2.getTrackbarPos('C','image')  
    blur = cv2.getTrackbarPos('Blur','image')
    s  = cv2.getTrackbarPos(switch,'image')
    
    img_blur   = cv2.GaussianBlur(img, (2*blur+1 ,2*blur+1), 0)
    img_gray   = cv2.cvtColor(img_blur, cv2.COLOR_BGR2GRAY)
    if s == 0:
        th = img_blur
    else:
        if bs%2 == 0:
            bs += 3
        th = get_th_ad1(img_gray,bs,c)
        

cv2.destroyAllWindows()
print("{},{}".format(bs,c))

0,0
